<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель <br>Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Домашнее задание № 8. Часть 2
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План 2 части домашнего задания
    2.1. Введение
    2.2. Описание данных
    2.3. Предобработка данных
    2.4. Обучение и проверка моделей
    2.5. Заключение

### 2.1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://goo.gl/forms/z8zENbMiaEAeB7nG3) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в README [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2.2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://yadi.sk/d/krikdUic3Ggjyf) эту выборку. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [5]:
!head -1 stackoverflow.10kk.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [6]:
%%time
!wc -l stackoverflow.10kk.tsv

10000000 stackoverflow.10kk.tsv
CPU times: user 67 ms, sys: 10 ms, total: 77 ms
Wall time: 3.74 s


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 2.3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [3]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

Должно получиться вот такое число строк – 4389054. Как видите, 10 Гб у меня обработались примерно за полторы минуты.

In [46]:
%%time
!python preprocess.py -i stackoverflow.10kk.tsv -o stackoverflow.vw

0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
CPU times: user 11.3 s, sys: 1.63 s, total: 12.9 s
Wall time: 10min 48s


In [47]:
!wc -l stackoverflow.vw

4522552 stackoverflow.vw


Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [49]:
4522552 / 3

1507517

In [51]:
!head -1507517 stackoverflow.vw > stackoverflow_train.vw

In [52]:
!tail -1507517 stackoverflow.vw > stackoverflow_test.vw

In [55]:
!head -3015035 stackoverflow.vw | tail -1507518 > stackoverflow_valid.vw

In [56]:
!wc -l stackoverflow_train.vw

1507517 stackoverflow_train.vw


In [57]:
!wc -l stackoverflow_test.vw

1507517 stackoverflow_test.vw


In [58]:
!wc -l stackoverflow_valid.vw

1507518 stackoverflow_valid.vw


In [60]:
!cut --d '|' -f1 stackoverflow_valid.vw > stackoverflow_valid_labels.txt

In [61]:
!cut --d '|' -f1 stackoverflow_test.vw > stackoverflow_test_labels.txt

### 2.4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: bit_precision=28 и seed=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [109]:
import os
from sklearn.metrics import accuracy_score
with open('stackoverflow_valid_labels.txt') as true_file:
    true_labels = [int(label) for label in true_file.readlines()]

In [154]:
%%time
results = {}
for pas in [1,3,5]:
    for ngram in [1,2,3]:
        print "Running with %s passes and %s ngrams" % (pas, ngram)
        vw_params_train = "--oaa 10 -d stackoverflow_train.vw -f models/model_%spass_%sngram --loss_function=hinge --passes %s -c --ngram %s -b 28 --random_seed 17" % (pas, ngram, pas, ngram)
        vw_params_test = "-i models/model_%spass_%sngram -t -d stackoverflow_valid.vw -p preds/pred_%spass_%sngram.txt" % (pas, ngram, pas, ngram)
        
        vw_train = os.system("vw " + vw_params_train)
        vw_test = os.system("vw " + vw_params_test)
        
        with open('preds/pred_%spass_%sngram.txt' % (pas, ngram)) as pred_file:
            valid_prediction = [int(label) for label in pred_file.readlines()]
            
        key = "%spass_%sngram" % (pas, ngram)
        results[key] = accuracy_score(valid_prediction, true_labels)

Running with 1 passes and 1 ngrams
Running with 1 passes and 2 ngrams
Running with 1 passes and 3 ngrams
Running with 3 passes and 1 ngrams
Running with 3 passes and 2 ngrams
Running with 3 passes and 3 ngrams
Running with 5 passes and 1 ngrams
Running with 5 passes and 2 ngrams
Running with 5 passes and 3 ngrams
CPU times: user 14.3 s, sys: 134 ms, total: 14.4 s
Wall time: 46min 1s


In [160]:
results

{'1pass_1ngram': 0.9006320322543413,
 '1pass_2ngram': 0.91761226068279123,
 '1pass_3ngram': 0.91482556095515943,
 '3pass_1ngram': 0.90172256649671845,
 '3pass_2ngram': 0.9146318650921581,
 '3pass_3ngram': 0.91191348959017404,
 '5pass_1ngram': 0.90102406737431995,
 '5pass_2ngram': 0.91573765620045666,
 '5pass_3ngram': 0.91259407847866492}

In [159]:
max(results.values())

0.91761226068279123

### 1 pass 1 ngram

In [149]:
%%time
## 1 pass 1 ngram
!vw --oaa 10 -d stackoverflow_train.vw -f models/model_3pass_1ngram \
--loss_function=hinge --passes 3 -c --ngram 1 -b 28 --random_seed 17

Generating 1-grams for all namespaces.
final_regressor = models/model_3pass_1ngram
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = stackoverflow_train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      161
0.500000 1.000000            2            2.0        4        1       68
0.750000 1.000000            4            4.0        7        1       88
0.750000 0.750000            8            8.0        7        1       95
0.750000 0.750000           16           16.0        2        7      159
0.750000 0.750000           32           32.0        1        7      404
0.734375 0.718750           64           64.0        5        1       54
0.664062 0.593750          128          128.0        6     

In [150]:
!vw -i models/model_3pass_1ngram -t -d stackoverflow_valid.vw \
-p preds/pred_3pass_1ngram.txt

Generating 1-grams for all namespaces.
only testing
predictions = preds/pred_3pass_1ngram.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = stackoverflow_valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        2        2       72
0.000000 0.000000            2            2.0        1        1      143
0.000000 0.000000            4            4.0        7        7      180
0.000000 0.000000            8            8.0        5        5      116
0.062500 0.125000           16           16.0        7        7      246
0.093750 0.125000           32           32.0        2        2      201
0.187500 0.281250           64           64.0        1        1       65
0.164062 0.140625          128          128.0        7        1       22
0.128906 0.093750          256    

In [151]:
with open('preds/pred_3pass_1ngram.txt') as pred_file:
    valid_prediction = [int(label) for label in pred_file.readlines()]

In [152]:
print("Accuracy: {}".format(accuracy_score(valid_prediction, true_labels)))

Accuracy: 0.901722566497


<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- Биграммы и 1 проход по выборке
- Униграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [161]:
!vw -i models/model_1pass_2ngram -t -d stackoverflow_test.vw -p preds/pred_best_model_test.txt

Generating 2-grams for all namespaces.
only testing
predictions = preds/pred_best_model_test.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = stackoverflow_test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        5        5      466
0.000000 0.000000            2            2.0        1        1      264
0.000000 0.000000            4            4.0        3        3      504
0.000000 0.000000            8            8.0        6        6      346
0.000000 0.000000           16           16.0        5        5      350
0.000000 0.000000           32           32.0        1        1      320
0.062500 0.125000           64           64.0        7        7      520
0.093750 0.125000          128          128.0        4        4      148
0.082031 0.070312          256  

In [162]:
with open('preds/pred_best_model_test.txt') as pred_file:
    valid_prediction = [int(label) for label in pred_file.readlines()]

In [164]:
with open('stackoverflow_test_labels.txt') as true_file:
    true_labels_test = [int(label) for label in true_file.readlines()]

In [165]:
print("Accuracy: {}".format(accuracy_score(valid_prediction, true_labels_test)))

Accuracy: 0.917525308172


In [ ]:
## validation Accuracy: 91.761226068279123
## test Accuracy: 91.7525308172

<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- Результаты почти одинаковы – отличаются меньше чем на 0.5%

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [167]:
!cat stackoverflow_train.vw stackoverflow_valid.vw > stackoverflow_big_train.vw

In [168]:
%%time
## 1 pass 2 ngram
!vw --oaa 10 -d stackoverflow_big_train.vw -f models/final_model_1pass_2ngram \
--loss_function=hinge --passes 1 -c --ngram 2 -b 28 --random_seed 17

Generating 2-grams for all namespaces.
final_regressor = models/final_model_1pass_2ngram
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = stackoverflow_big_train.vw.cache
Reading datafile = stackoverflow_big_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      320
0.500000 1.000000            2            2.0        4        1      134
0.750000 1.000000            4            4.0        7        1      174
0.750000 0.750000            8            8.0        7        1      188
0.750000 0.750000           16           16.0        7        7      416
0.781250 0.812500           32           32.0        7        2      346
0.734375 0.687500           64           64.0        7        2      248
0.640625 0.546875          128          128.0        2        2     

In [169]:
!vw -i models/final_model_1pass_2ngram -t -d stackoverflow_test.vw -p preds/pred_final_model_test.txt

Generating 2-grams for all namespaces.
only testing
predictions = preds/pred_final_model_test.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = stackoverflow_test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        5        5      466
0.000000 0.000000            2            2.0        1        1      264
0.000000 0.000000            4            4.0        3        3      504
0.000000 0.000000            8            8.0        6        6      346
0.000000 0.000000           16           16.0        5        5      350
0.000000 0.000000           32           32.0        1        1      320
0.062500 0.125000           64           64.0        7        7      520
0.062500 0.062500          128          128.0        4        4      148
0.046875 0.031250          256 

In [170]:
with open('preds/pred_final_model_test.txt') as pred_file:
    valid_prediction = [int(label) for label in pred_file.readlines()]

In [171]:
print("Accuracy: {}".format(accuracy_score(valid_prediction, true_labels_test)))

Accuracy: 0.921840350722


In [ ]:
# Было Accuracy: 0.917525308172

In [172]:
92.1840350722 - 91.7525308172

0.4315042550000072

<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - 0.4%
 - 0.8%
 - 1.2%

### 2.5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)